In [4]:
import pandas as pd
from torch import nn, optim
from sklearn.model_selection import train_test_split
import torch

ratings = pd.read_csv('../data/lens_tmdb/ratings_small.csv')
ratings['userId'] = ratings['userId'].astype('category').cat.codes.values
ratings['movieId'] = ratings['movieId'].astype('category').cat.codes.values
train, valid = train_test_split(ratings, test_size=0.2)




In [8]:
class RecommenderLSTM(nn.Module):
    def __init__(self, n_users, n_movies, n_factors, hidden_size):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.movie_emb = nn.Embedding(n_movies, n_factors)
        self.lstm = nn.LSTM(n_factors * 2, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, users, movies):
        user_emb = self.user_emb(users)
        movie_emb = self.movie_emb(movies)
        x = torch.cat([user_emb, movie_emb], dim=1)
        x, _ = self.lstm(x.unsqueeze(1))
        x = self.fc(x.squeeze(1))
        return x

n_users = len(ratings['userId'].unique())
n_movies = len(ratings['movieId'].unique())
n_factors = 20
hidden_size = 32
model = RecommenderLSTM(n_users, n_movies, n_factors, hidden_size)


In [9]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for _, row in train.iterrows():
        user = torch.LongTensor([int(row['userId'])]).to(device)
        movie = torch.LongTensor([int(row['movieId'])]).to(device)
        rating = torch.FloatTensor([float(row['rating'])]).to(device)

        optimizer.zero_grad()
        prediction = model(user, movie)
        loss = criterion(prediction, rating)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {total_loss/len(train)}")


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR